In [1]:

from __future__ import print_function
from collections import defaultdict
import os
import sys
import posixpath

from elftools.elf.elffile import ELFFile

import os
import json 

import sys,os



In [2]:
# from transformers import AutoTokenizer

import torch
from transformers import AutoTokenizer



# old_tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
# old_tokenizer.pad_token
from transformers import BartForConditionalGeneration, BartTokenizer

# old_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
old_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
# tok = BartTokenizer.from_pretrained("facebook/bart-large")
# example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
# batch = tok(example_english_phrase, return_tensors="pt")
# generated_ids = model.generate(batch["input_ids"])
# assert tok.batch_decode(generated_ids, skip_special_tokens=True) == [
#     "UN Chief Says There Is No Plan to Stop Chemical Weapons in Syria"
# ]


In [3]:
sep_token = old_tokenizer.sep_token
pad_token = old_tokenizer.pad_token
cls_token = old_tokenizer.cls_token

print(sep_token , pad_token , cls_token)

</s> <pad> <s>


In [4]:
MAX_TOKEN_LEN = 1024



# disassembly_decimal disassembly_all_number_to_words disassembly_decimal  disassembly_addresses_to_words
data_key = "disassembly_decimal"





# def generate_param_string(param_list):


#     if len(param_list)==0:
#         return 'void'
#     else:
#         return " ".join(param_list)
def unique_items(list_of_lists):
    # Flatten the list of lists
    print(list_of_lists[3].split())
    flattened_list = [item for sublist in list_of_lists for item in sublist.split()]
    # Convert the flattened list to a set to remove duplicates
    unique_elements = set(flattened_list)
    return list(unique_elements)






DATA_PATH = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions'

TRAIN_DATA_PATH  ='/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/train/'

TEST_DATA_PATH   = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/test/'




train_json_files = [os.path.join(TRAIN_DATA_PATH, f) for f in os.listdir(TRAIN_DATA_PATH) ]

test_json_files = [os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH) ]




all_types = ['*short', '*ulong', 'int', 'ulonglong', '*longlong', 'None', 'short', 'char', '*struct', '*long', '*char', 'double', 'uint', 'uchar', '*float', 'longlong', 'long', '*void', 'float', 'bool', '*int', '*ushort', '*ulonglong', '*uchar', 'ushort', 'void', '*double', 'ulong', '*uint', '*bool']
all_types_dict = {}
for k,type in enumerate(all_types):
    all_types_dict[type] = str(k)

def read_corpus(json_files):

    all = []
    all_params = []
    all_returns = []
    all_signatures = []

    for k, j_file in enumerate(json_files):
        # if k>1000:
        #     break
        try:

            with open(j_file, 'r') as file:
                data = json.load(file)

                funct = data[data_key]['input']
                
                all.append(funct)

                param_string = ""

                if len(data['signature']['paramTypes'])==0:
                    param_string = all_types_dict['void']
                    
                for v,type in enumerate(data['signature']['paramTypes']):
                    if v==0:
                        param_string =  all_types_dict[type]
                    else:
                        param_string = param_string + sep_token +all_types_dict[type]
                
                all_params.append( param_string )
                all_returns.append(  data['signature']['returnType']  )
                all_signatures.append(param_string + sep_token + all_types_dict[data['signature']['returnType']] )
        except Exception as e: 
            print(e)
    return all , all_params , all_returns , all_signatures
    


train_text , train_params , train_returns, train_signatures = read_corpus(train_json_files)

test_text  , test_params  , test_returns , test_signatures = read_corpus(test_json_files)



        
# text = text[0:5000]
print("Functions Count: ",len(train_text), '\n')
example = train_text[10]
text = train_text + test_text
signatures = train_signatures + test_signatures
input_output_data = []
for i in range(len(text)):
    input_output_data.append(text[i])
    input_output_data.append(signatures[i])
    
unique_types = unique_items(train_params)
print(len(unique_types))




def get_training_corpus():
    for i in range(0, len(text), 1000):
        yield input_output_data[i : i + 1000]


Functions Count:  80000 

['2</s>8']
1566


In [5]:

tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus() , vocab_size =old_tokenizer.vocab_size)
special_tokens = {"additional_special_tokens": ["\n" , ' ']}
tokenizer.add_special_tokens(special_tokens)

2

In [6]:
tokenizer.save_pretrained("./../models/bart_ASIS")

('./../models/bart_ASIS/tokenizer_config.json',
 './../models/bart_ASIS/special_tokens_map.json',
 './../models/bart_ASIS/vocab.json',
 './../models/bart_ASIS/merges.txt',
 './../models/bart_ASIS/added_tokens.json',
 './../models/bart_ASIS/tokenizer.json')

In [7]:
signatures[1]

'25</s>2'